# Experimental Notebook
This notebook serves as a dump for experimentation for tables in DuckDB. 

## Import packages

In [1]:
# install packages and establish database connections
import duckdb
import geopandas as gpd
from shapely import wkt

conn = duckdb.connect("../data/lake/database.duckdb")

## List available tables

In [4]:
# show all available schemas and tables
conn.sql("""
    SELECT table_schema, table_name
    FROM information_schema.tables
    WHERE table_schema NOT IN ('information_schema', 'pg_catalog')
    ORDER BY table_schema, table_name;
""").pl()

table_schema,table_name
str,str
"""public""","""climate_atmosphere_raw"""
"""public""","""disease_pidsr_totals_raw"""
"""public""","""location_raw"""
"""public""","""nasa_firms_raw"""
"""public""","""project_cchain_climate_atmosph…"
"""public""","""project_cchain_climate_atmosph…"
"""public""","""project_cchain_disease_pidsr_t…"
"""public""","""project_cchain_disease_pidsr_t…"
"""public""","""waqi_airquality_raw"""


## NASA FIRMS

In [38]:
# sample query
conn.sql("""
    SELECT *
    FROM public.nasa_firms_raw
    ORDER BY acq_date, acq_time
    """).pl()

____S______X_S__country_id,latitude,longitude,bright_ti4,scan,track,acq_date,acq_time,satellite,instrument,confidence,version,bright_ti5,frp,daynight,measurement_date
str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,date
"""THA""","""18.84668""","""100.32352""","""309.01""","""0.41""","""0.37""","""2025-02-01""","""1847""","""N""","""VIIRS""","""n""","""2.0NRT""","""290.88""","""0.88""","""N""",2025-02-01
"""THA""","""18.85875""","""101.03607""","""308.25""","""0.39""","""0.36""","""2025-02-01""","""1847""","""N""","""VIIRS""","""n""","""2.0NRT""","""288.84""","""0.92""","""N""",2025-02-01
"""THA""","""18.87356""","""99.68552""","""310.19""","""0.42""","""0.38""","""2025-02-01""","""1847""","""N""","""VIIRS""","""n""","""2.0NRT""","""290.93""","""1.56""","""N""",2025-02-01
"""THA""","""18.87704""","""99.68604""","""303.29""","""0.42""","""0.38""","""2025-02-01""","""1847""","""N""","""VIIRS""","""n""","""2.0NRT""","""290.43""","""0.58""","""N""",2025-02-01
"""THA""","""18.89662""","""100.22108""","""301.68""","""0.41""","""0.37""","""2025-02-01""","""1847""","""N""","""VIIRS""","""n""","""2.0NRT""","""290.79""","""0.46""","""N""",2025-02-01
…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…
"""THA""","""19.96813""","""100.15673""","""332.3""","""0.44""","""0.39""","""2025-04-26""","""644""","""N""","""VIIRS""","""l""","""2.0NRT""","""303.14""","""1.28""","""D""",2025-04-26
"""THA""","""19.97167""","""100.15617""","""334.31""","""0.44""","""0.39""","""2025-04-26""","""644""","""N""","""VIIRS""","""n""","""2.0NRT""","""302.34""","""2.26""","""D""",2025-04-26
"""THA""","""19.99866""","""100.3531""","""335.39""","""0.45""","""0.39""","""2025-04-26""","""644""","""N""","""VIIRS""","""n""","""2.0NRT""","""302.86""","""5.03""","""D""",2025-04-26


In [13]:
# sample query
df = conn.sql("""
    SELECT
        CAST(____S______X_S__country_id AS VARCHAR) AS country_id,
        CAST(latitude AS FLOAT) AS latitude,
        CAST(longitude AS FLOAT) AS longitude,
        CONCAT('POINT (', longitude, ' ', latitude,')') AS geometry,
        CAST(bright_ti4 AS FLOAT) AS bright_ti4,
        CAST(scan AS FLOAT) AS scan,
        CAST(track AS FLOAT) AS track,
        STRPTIME(CONCAT(acq_date, ' ',
            CASE
                WHEN LENGTH(acq_time) = 4 THEN CONCAT(SUBSTR(acq_time, 1, 2), ':', SUBSTR(acq_time, 3, 2), ':00')
                WHEN LENGTH(acq_time) = 3 THEN CONCAT('0', SUBSTR(acq_time, 1, 1), ':', SUBSTR(acq_time, 2, 2), ':00')
                WHEN LENGTH(acq_time) = 2 THEN CONCAT('00:', acq_time, ':00')
                ELSE NULL
            END),
            '%Y-%m-%d %H:%M:%S'
        ) AS acq_datetime,
        CAST(satellite AS VARCHAR) AS satellite,
        CAST(instrument AS VARCHAR) AS instrument,
        CAST(confidence AS VARCHAR) AS confidence,
        CAST(version AS VARCHAR) AS version,
        CAST(bright_ti5 AS FLOAT) AS bright_ti5,
        CAST(frp AS FLOAT) AS frp,
        CAST(daynight AS VARCHAR) AS daynight,
        CAST(measurement_date AS DATE) AS measurement_date
    FROM public.nasa_firms_raw
    ORDER BY acq_datetime;
    """).pl()

# convert to pandas
df = df.to_pandas()

# create geometries in geodataframe
df["geometry"] = df["geometry"].apply(wkt.loads)
gpd.GeoDataFrame(df, geometry="geometry")

,country_id,latitude,longitude,geometry,bright_ti4,scan,track,acq_datetime,satellite,instrument,confidence,version,bright_ti5,frp,daynight,measurement_date
0,THA,7.770230,99.198730,POINT (99.19873 7.77023),334.279999,0.53,0.50,2025-02-01 06:16:00,N,VIIRS,n,2.0NRT,297.679993,3.27,D,2025-02-01
1,THA,7.789710,99.313652,POINT (99.31365 7.78971),343.109985,0.52,0.50,2025-02-01 06:16:00,N,VIIRS,n,2.0NRT,300.500000,10.58,D,2025-02-01
2,THA,9.157640,99.054543,POINT (99.05454 9.15764),331.929993,0.51,0.50,2025-02-01 06:16:00,N,VIIRS,n,2.0NRT,297.540009,4.03,D,2025-02-01
3,THA,9.351550,98.980347,POINT (98.98035 9.35155),328.570007,0.52,0.50,2025-02-01 06:16:00,N,VIIRS,n,2.0NRT,297.089996,3.33,D,2025-02-01
4,THA,12.521200,99.823509,POINT (99.82351 12.5212),348.799988,0.41,0.45,2025-02-01 06:16:00,N,VIIRS,n,2.0NRT,297.950012,3.48,D,2025-02-01
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
77807,THA,19.737671,100.326523,POINT (100.32652 19.73767),304.170013,0.51,0.41,2025-04-26 19:13:00,N,VIIRS,n,2.0NRT,278.230011,1.47,N,2025-04-26
77808,THA,19.744410,100.329308,POINT (100.32931 19.74441),300.359985,0.51,0.41,2025-04-26 19:13:00,N,VIIRS,n,2.0NRT,277.100006,1.06,N,2025-04-26
77809,THA,19.799959,100.366730,POINT (100.36673 19.79996),302.489990,0.51,0.41,2025-04-26 19:13:00,N,VIIRS,n,2.0NRT,277.260010,1.01,N,2025-04-26
77810,THA,8.562080,98.974892,POINT (98.97489 8.56208),308.149994,0.42,0.45,2025-04-26 19:15:00,N,VIIRS,n,2.0NRT,289.100006,0.72,N,2025-04-26


## WAQI Air Quality

In [5]:
# sample query
conn.sql("""
    SELECT *
    FROM public.waqi_airquality_raw
    ORDER BY date
    LIMIT 50;
    """).pl()

date,pm25,pm10,o3,no2,so2,co
str,str,str,str,str,str,str
"""01-Apr-14""",null,"""89""","""12""","""33""","""28""","""15"""
"""01-Apr-15""",null,"""45""","""48""","""14""","""14""","""9"""
"""01-Apr-16""","""134""","""72""","""54""","""27""","""1""","""14"""
"""01-Apr-17""","""145""","""35""","""17""","""5""",""" """,""" """
"""01-Apr-18""","""135""","""64""","""30""","""6""","""11""",""" """
…,…,…,…,…,…,…
"""01-Jan-14""",null,"""56""","""43""","""20""","""3""","""9"""
"""01-Jan-15""",null,"""39""","""37""","""15""","""9""","""10"""
"""01-Jan-17""",null,"""22""","""12""","""4""",""" """,""" """


## Project CCHAIN

In [5]:
# sample query
conn.sql("""
    SELECT *
    FROM public.project_cchain_climate_atmosphere_raw
    ORDER BY date
    LIMIT 50;
    """).pl()

FloatProgress(value=0.0, layout=Layout(width='auto'), style=ProgressStyle(bar_color='black'))

uuid,date,adm1_en,adm1_pcode,adm2_en,adm2_pcode,adm3_en,adm3_pcode,adm4_en,adm4_pcode,tave,tmin,tmax,heat_index,pr,wind_speed,rh,solar_rad,uv_rad
str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str
"""CATMS039120""","""2003-01-01""","""Region III""","""PH030000000""","""Nueva Ecija""","""PH034900000""","""Palayan City""","""PH034919000""","""Bo. Militar (Fort Magsaysay)""","""PH034919003""","""24.74""","""21.27""","""29.01""","""25.51""","""0.0""","""3.13""","""73.67""","""203.03""","""24.45"""
"""CATMS01E519""","""2003-01-01""","""Region I""","""PH010000000""","""Pangasinan""","""PH015500000""","""Dagupan City""","""PH015518000""","""Mamalingling""","""PH015518019""","""26.85""","""24.49""","""30.17""","""27.91""","""0.0""","""2.0""","""68.59""","""193.62""","""23.77"""
"""CATMS021E2B""","""2003-01-01""","""Region I""","""PH010000000""","""Pangasinan""","""PH015500000""","""Dagupan City""","""PH015518000""","""Mayombo""","""PH015518021""","""26.85""","""24.49""","""30.17""","""27.91""","""0.0""","""2.0""","""68.59""","""193.62""","""23.77"""
"""CATMS0458DF""","""2003-01-01""","""Region III""","""PH030000000""","""Nueva Ecija""","""PH034900000""","""Palayan City""","""PH034919000""","""Sapang Buho""","""PH034919011""","""24.74""","""21.27""","""29.01""","""25.51""","""0.0""","""3.13""","""73.67""","""203.03""","""24.45"""
"""CATMS00C7BF""","""2003-01-01""","""Region I""","""PH010000000""","""Pangasinan""","""PH015500000""","""Dagupan City""","""PH015518000""","""Bonuan Gueset""","""PH015518009""","""26.85""","""24.49""","""30.17""","""27.91""","""0.0""","""2.0""","""68.59""","""193.62""","""23.77"""
…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…
"""CATMS0559B0""","""2003-01-01""","""Region III""","""PH030000000""","""Nueva Ecija""","""PH034900000""","""Palayan City""","""PH034919000""","""Popolon Pagas""","""PH034919021""","""25.66""","""21.46""","""30.74""","""26.54""","""0.0""","""2.81""","""70.87""","""209.25""","""25.13"""
"""CATMS01C890""","""2003-01-01""","""Region I""","""PH010000000""","""Pangasinan""","""PH015500000""","""Dagupan City""","""PH015518000""","""Malued""","""PH015518018""","""26.85""","""24.49""","""30.17""","""27.91""","""0.0""","""2.0""","""68.59""","""193.62""","""23.77"""
"""CATMS00AB36""","""2003-01-01""","""Region I""","""PH010000000""","""Pangasinan""","""PH015500000""","""Dagupan City""","""PH015518000""","""Bonuan Boquig""","""PH015518008""","""26.85""","""24.49""","""30.17""","""27.91""","""0.0""","""2.0""","""68.59""","""193.62""","""23.77"""


In [18]:
# sample query
conn.sql("""
    SELECT
        CAST(uuid AS VARCHAR) AS uuid,
        CAST(date AS DATE) AS date,
        CAST(adm1_en AS VARCHAR) AS adm1_en,
        CAST(adm1_pcode AS VARCHAR) AS adm1_pcode,
        CAST(adm2_en AS VARCHAR) AS adm2_en,
        CAST(adm2_pcode AS VARCHAR) AS adm2_pcode,
        CAST(adm3_en AS VARCHAR) AS adm3_en,
        CAST(adm3_pcode AS VARCHAR) AS adm3_pcode,
        CAST(adm4_en AS VARCHAR) AS adm4_en,
        CAST(adm4_pcode AS VARCHAR) AS adm4_pcode,
        COALESCE(CAST(tave AS FLOAT), 0) AS tave,
        COALESCE(CAST(tmin AS FLOAT), 0) AS tmin,
        COALESCE(CAST(tmax AS FLOAT), 0) AS tmax,
        COALESCE(CAST(heat_index AS FLOAT), 0) AS heat_index,
        COALESCE(CAST(pr AS FLOAT), 0) AS pr,
        COALESCE(CAST(wind_speed AS FLOAT), 0) AS wind_speed,
        COALESCE(CAST(rh AS FLOAT), 0) AS rh,
        COALESCE(CAST(solar_rad AS FLOAT), 0) AS solar_rad,
        COALESCE(CAST(uv_rad AS FLOAT), 0) AS uv_rad
    FROM public.project_cchain_climate_atmosphere_raw
    ORDER BY pr ASC
    LIMIT 50;
    """).pl()

uuid,date,adm1_en,adm1_pcode,adm2_en,adm2_pcode,adm3_en,adm3_pcode,adm4_en,adm4_pcode,tave,tmin,tmax,heat_index,pr,wind_speed,rh,solar_rad,uv_rad
str,date,str,str,str,str,str,str,str,str,f32,f32,f32,f32,f32,f32,f32,f32,f32
"""CATMS339D3C""",2004-01-09,"""Region VIII""","""PH080000000""","""Leyte""","""PH083700000""","""Tacloban City""","""PH083747000""","""Barangay 23-A""","""PH083747154""",26.57,24.52,29.23,27.690001,0.0,4.7,68.760002,229.410004,26.66
"""CATMS33642A""",2004-01-09,"""Region VIII""","""PH080000000""","""Leyte""","""PH083700000""","""Tacloban City""","""PH083747000""","""Barangay 95-A (Caibaan)""","""PH083747152""",26.57,24.52,29.23,27.690001,0.0,4.7,68.760002,229.410004,26.66
"""CATMS33B9C5""",2004-01-09,"""Region VIII""","""PH080000000""","""Leyte""","""PH083700000""","""Tacloban City""","""PH083747000""","""Barangay 94-A""","""PH083747155""",26.57,24.52,29.23,27.690001,0.0,4.7,68.760002,229.410004,26.66
"""CATMS340F60""",2004-01-09,"""Region IX""","""PH090000000""","""Zamboanga del Sur""","""PH097300000""","""Zamboanga City""","""PH097332000""","""Baliwasan""","""PH097332004""",25.940001,24.940001,27.370001,27.23,0.0,3.05,78.620003,264.859985,30.82
"""CATMS33D64E""",2004-01-09,"""Region IX""","""PH090000000""","""Zamboanga del Sur""","""PH097300000""","""Zamboanga City""","""PH097332000""","""Arena Blanco""","""PH097332001""",26.25,25.67,26.950001,27.91,0.0,3.19,77.379997,266.200012,30.77
…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…
"""CATMS369FAF""",2004-01-09,"""Region IX""","""PH090000000""","""Zamboanga del Sur""","""PH097300000""","""Zamboanga City""","""PH097332000""","""La Paz""","""PH097332032""",25.860001,24.51,27.73,27.120001,0.0,3.37,78.129997,265.309998,30.780001
"""CATMS34D71F""",2004-01-09,"""Region IX""","""PH090000000""","""Zamboanga del Sur""","""PH097300000""","""Zamboanga City""","""PH097332000""","""Cabaluay""","""PH097332015""",26.25,25.67,26.950001,27.91,0.0,3.19,77.379997,266.200012,30.77
"""CATMS33F2D7""",2004-01-09,"""Region IX""","""PH090000000""","""Zamboanga del Sur""","""PH097300000""","""Zamboanga City""","""PH097332000""","""Ayala""","""PH097332002""",25.940001,24.940001,27.370001,27.23,0.0,3.05,78.620003,264.859985,30.82


In [5]:
conn.sql("""
    SELECT *
    FROM public.project_cchain_climate_atmosphere_raw
    WHERE pr != 0
    LIMIT 50
    """).pl()

FloatProgress(value=0.0, layout=Layout(width='auto'), style=ProgressStyle(bar_color='black'))

uuid,date,adm1_en,adm1_pcode,adm2_en,adm2_pcode,adm3_en,adm3_pcode,adm4_en,adm4_pcode,tave,tmin,tmax,heat_index,pr,wind_speed,rh,solar_rad,uv_rad
str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str
"""CATMS0592C2""","""2003-01-01""","""Region V""","""PH050000000""","""Albay""","""PH050500000""","""Legazpi City""","""PH050506000""","""Bgy. 47 - Arimbay""","""PH050506001""","""26.0""","""23.76""","""28.66""","""27.95""","""14.03""","""4.16""","""83.32""","""144.71""","""17.33"""
"""CATMS05AF4B""","""2003-01-01""","""Region V""","""PH050000000""","""Albay""","""PH050500000""","""Legazpi City""","""PH050506000""","""Bgy. 64 - Bagacay (Bgy. 41 Bag…","""PH050506002""","""26.21""","""24.46""","""28.24""","""28.15""","""10.63""","""5.2""","""82.01""","""154.44""","""18.02"""
"""CATMS05CBD4""","""2003-01-01""","""Region V""","""PH050000000""","""Albay""","""PH050500000""","""Legazpi City""","""PH050506000""","""Bgy. 48 - Bagong Abre (Bgy. 42…","""PH050506003""","""26.0""","""23.76""","""28.66""","""27.95""","""14.24""","""4.16""","""83.32""","""144.71""","""17.33"""
"""CATMS05E85D""","""2003-01-01""","""Region V""","""PH050000000""","""Albay""","""PH050500000""","""Legazpi City""","""PH050506000""","""Bgy. 66 - Banquerohan (Bgy. 43…","""PH050506004""","""26.21""","""24.46""","""28.24""","""28.15""","""10.76""","""5.2""","""82.01""","""154.44""","""18.02"""
"""CATMS0604E6""","""2003-01-01""","""Region V""","""PH050000000""","""Albay""","""PH050500000""","""Legazpi City""","""PH050506000""","""Bgy. 1 - Em's Barrio (Pob.)""","""PH050506005""","""26.0""","""23.76""","""28.66""","""27.95""","""12.72""","""4.16""","""83.32""","""144.71""","""17.33"""
…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…
"""CATMS0A96D7""","""2003-01-01""","""Region V""","""PH050000000""","""Albay""","""PH050500000""","""Legazpi City""","""PH050506000""","""Bgy. 70 - Cagbacong (Bgy. 50)""","""PH050506050""","""26.21""","""24.46""","""28.24""","""28.15""","""10.72""","""5.2""","""82.01""","""154.44""","""18.02"""
"""CATMS0AB360""","""2003-01-01""","""Region V""","""PH050000000""","""Albay""","""PH050500000""","""Legazpi City""","""PH050506000""","""Bgy. 40 - Cruzada (Bgy. 52)""","""PH050506051""","""26.0""","""23.76""","""28.66""","""27.95""","""12.72""","""4.16""","""83.32""","""144.71""","""17.33"""
"""CATMS0ACFE9""","""2003-01-01""","""Region V""","""PH050000000""","""Albay""","""PH050500000""","""Legazpi City""","""PH050506000""","""Bgy. 57 - Dap-dap (Bgy. 69)""","""PH050506052""","""26.0""","""23.76""","""28.66""","""27.95""","""11.16""","""4.16""","""83.32""","""144.71""","""17.33"""


In [5]:
# sample query
conn.sql("""
    SELECT
        CAST(uuid AS VARCHAR) AS uuid,
        CAST(date AS DATE) AS date,
        CAST(adm1_en AS VARCHAR) AS adm1_en,
        CAST(adm1_pcode AS VARCHAR) AS adm1_pcode,
        CAST(adm2_en AS VARCHAR) AS adm2_en,
        CAST(adm2_pcode AS VARCHAR) AS adm2_pcode,
        CAST(adm3_en AS VARCHAR) AS adm3_en,
        CAST(adm3_pcode AS VARCHAR) AS adm3_pcode,
        CAST(disease_icd10_code AS VARCHAR) AS disease_icd10_code,
        CAST(disease_common_name AS VARCHAR) AS disease_common_name,
        CAST(case_total AS INT) AS case_total
    FROM public.project_cchain_disease_pidsr_totals_raw
    WHERE case_total != 0
    ORDER BY date, adm3_pcode
    """).pl()

FloatProgress(value=0.0, layout=Layout(width='auto'), style=ProgressStyle(bar_color='black'))

uuid,date,adm1_en,adm1_pcode,adm2_en,adm2_pcode,adm3_en,adm3_pcode,disease_icd10_code,disease_common_name,case_total
str,date,str,str,str,str,str,str,str,str,i32
"""DPSRT006DB0""",2008-01-07,"""Region I""","""PH010000000""","""Pangasinan""","""PH015500000""","""Dagupan City""","""PH015518000""","""A90-A91""","""DENGUE FEVER""",9
"""DPSRT006DB0""",2008-01-07,"""Region I""","""PH010000000""","""Pangasinan""","""PH015500000""","""Dagupan City""","""PH015518000""","""A90-A91""","""DENGUE FEVER""",9
"""DPSRT006DB0""",2008-01-07,"""Region I""","""PH010000000""","""Pangasinan""","""PH015500000""","""Dagupan City""","""PH015518000""","""A90-A91""","""DENGUE FEVER""",9
"""DPSRT006DB0""",2008-01-07,"""Region I""","""PH010000000""","""Pangasinan""","""PH015500000""","""Dagupan City""","""PH015518000""","""A90-A91""","""DENGUE FEVER""",9
"""DPSRT006DB0""",2008-01-07,"""Region I""","""PH010000000""","""Pangasinan""","""PH015500000""","""Dagupan City""","""PH015518000""","""A90-A91""","""DENGUE FEVER""",9
…,…,…,…,…,…,…,…,…,…,…
"""DPSRT00923E""",2022-12-26,"""National Capital Region""","""PH130000000""","""NCR, Fourth District""","""PH137600000""","""City of Muntinlupa""","""PH137603000""","""A90-A91""","""DENGUE FEVER""",5
"""DPSRT00923E""",2022-12-26,"""National Capital Region""","""PH130000000""","""NCR, Fourth District""","""PH137600000""","""City of Muntinlupa""","""PH137603000""","""A90-A91""","""DENGUE FEVER""",5
"""DPSRT00923E""",2022-12-26,"""National Capital Region""","""PH130000000""","""NCR, Fourth District""","""PH137600000""","""City of Muntinlupa""","""PH137603000""","""A90-A91""","""DENGUE FEVER""",5
